In [1]:
!kubectl get nodes

NAME              STATUS   ROLES    AGE   VERSION
psnode66.ps.lab   Ready    master   73m   v1.14.1
psnode67.ps.lab   Ready    <none>   67m   v1.14.1
psnode68.ps.lab   Ready    <none>   67m   v1.14.1


In [2]:
!kubectl run kubernetes-bootcamp --image=gcr.io/google-samples/kubernetes-bootcamp:v1 --port=8080

kubectl run --generator=deployment/apps.v1 is DEPRECATED and will be removed in a future version. Use kubectl run --generator=run-pod/v1 or kubectl create instead.
deployment.apps/kubernetes-bootcamp created


In [3]:
!kubectl get pods

NAME                                  READY   STATUS              RESTARTS   AGE
kubernetes-bootcamp-b94cb9bff-b8xfb   0/1     ContainerCreating   0          13s


In [4]:
!kubectl get pods

NAME                                  READY   STATUS    RESTARTS   AGE
kubernetes-bootcamp-b94cb9bff-b8xfb   1/1     Running   0          3m23s


In [5]:
!kubectl describe pods

Name:               kubernetes-bootcamp-b94cb9bff-b8xfb
Namespace:          default
Priority:           0
PriorityClassName:  <none>
Node:               psnode68.ps.lab/10.20.30.68
Start Time:         Wed, 17 Apr 2019 11:48:01 -0700
Labels:             pod-template-hash=b94cb9bff
                    run=kubernetes-bootcamp
Annotations:        <none>
Status:             Running
IP:                 192.168.2.3
Controlled By:      ReplicaSet/kubernetes-bootcamp-b94cb9bff
Containers:
  kubernetes-bootcamp:
    Container ID:   docker://df94caee70e1c46ec1a79870f8580461f29c99e6c39e7b3f722706975364d37e
    Image:          gcr.io/google-samples/kubernetes-bootcamp:v1
    Image ID:       docker-pullable://gcr.io/google-samples/kubernetes-bootcamp@sha256:0d6b8ee63bb57c5f5b6156f446b3bc3b3c143d233037f3a2f00e279c8fcc64af
    Port:           8080/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Wed, 17 Apr 2019 11:48:29 -0700
    Ready:          True
    Restart Count:  0

In [6]:
%%sh
export POD_NAME=$(kubectl get pods -o go-template --template '{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}')
echo POD_NAME=${POD_NAME}

POD_NAME=kubernetes-bootcamp-b94cb9bff-b8xfb


In [7]:
#create a service
!kubectl expose deployment/kubernetes-bootcamp --type="NodePort" --port 8080

service/kubernetes-bootcamp exposed


In [9]:
!kubectl get deployments

NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp   1/1     1            1           4m45s


In [10]:
!kubectl get services

NAME                  TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
kubernetes            ClusterIP   10.96.0.1       <none>        443/TCP          78m
kubernetes-bootcamp   NodePort    10.107.224.35   <none>        8080:30484/TCP   70s


In [11]:
!kubectl describe services/kubernetes-bootcamp

Name:                     kubernetes-bootcamp
Namespace:                default
Labels:                   run=kubernetes-bootcamp
Annotations:              <none>
Selector:                 run=kubernetes-bootcamp
Type:                     NodePort
IP:                       10.107.224.35
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  30484/TCP
Endpoints:                192.168.2.3:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [17]:
%%sh
export NODE_PORT=$(kubectl get services/kubernetes-bootcamp -o go-template='{{(index .spec.ports 0).nodePort}}')
echo NODE_PORT=${NODE_PORT}
curl localhost:$NODE_PORT

NODE_PORT=30484
Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-b94cb9bff-b8xfb | v=1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    83    0    83    0     0   1397      0 --:--:-- --:--:-- --:--:--  1431


In [18]:
!kubectl get deployments

NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp   1/1     1            1           20m


In [19]:
#scale deployments
!kubectl scale deployments/kubernetes-bootcamp --replicas=4

deployment.extensions/kubernetes-bootcamp scaled


In [20]:
#right after scale command
!kubectl get deployments

NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp   2/4     4            2           20m


In [21]:
# a while later
!kubectl get deployments

NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp   4/4     4            4           21m


In [22]:
!kubectl get pods -o wide

NAME                                  READY   STATUS    RESTARTS   AGE   IP            NODE              NOMINATED NODE   READINESS GATES
kubernetes-bootcamp-b94cb9bff-b8xfb   1/1     Running   0          21m   192.168.2.3   psnode68.ps.lab   <none>           <none>
kubernetes-bootcamp-b94cb9bff-jcsgk   1/1     Running   0          59s   192.168.1.4   psnode67.ps.lab   <none>           <none>
kubernetes-bootcamp-b94cb9bff-pwf45   1/1     Running   0          58s   192.168.2.4   psnode68.ps.lab   <none>           <none>
kubernetes-bootcamp-b94cb9bff-vzrg5   1/1     Running   0          58s   192.168.1.5   psnode67.ps.lab   <none>           <none>


In [23]:
!kubectl describe deployments/kubernetes-bootcamp

Name:                   kubernetes-bootcamp
Namespace:              default
CreationTimestamp:      Wed, 17 Apr 2019 11:48:00 -0700
Labels:                 run=kubernetes-bootcamp
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               run=kubernetes-bootcamp
Replicas:               4 desired | 4 updated | 4 total | 4 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  run=kubernetes-bootcamp
  Containers:
   kubernetes-bootcamp:
    Image:        gcr.io/google-samples/kubernetes-bootcamp:v1
    Port:         8080/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Progressing    True    NewReplicaSetAvailable
  Available      True    MinimumReplicasAvailable
OldReplicaSets:  <none>
NewReplicaSet:   kubernetes-

In [24]:
!kubectl describe services/kubernetes-bootcamp

Name:                     kubernetes-bootcamp
Namespace:                default
Labels:                   run=kubernetes-bootcamp
Annotations:              <none>
Selector:                 run=kubernetes-bootcamp
Type:                     NodePort
IP:                       10.107.224.35
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  30484/TCP
Endpoints:                192.168.1.4:8080,192.168.1.5:8080,192.168.2.3:8080 + 1 more...
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [27]:
%%sh
export NODE_PORT=$(kubectl get services/kubernetes-bootcamp -o go-template='{{(index .spec.ports 0).nodePort}}')
echo NODE_PORT=$NODE_PORT
curl localhost:$NODE_PORT
curl localhost:$NODE_PORT
curl localhost:$NODE_PORT
curl localhost:$NODE_PORT
curl localhost:$NODE_PORT

NODE_PORT=30484
Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-b94cb9bff-b8xfb | v=1
Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-b94cb9bff-vzrg5 | v=1
Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-b94cb9bff-b8xfb | v=1
Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-b94cb9bff-vzrg5 | v=1
Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-b94cb9bff-jcsgk | v=1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    83    0    83    0     0   8477      0 --:--:-- --:--:-- --:--:--  9222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    83    0    83    0     0   1996      0 --:--:-- --:--:-- --:--:--  2024
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    83    0    83    0     0   6861      0 --:--:-- --:--:-- --:--:--  7545
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    83    0    83    0     0   5585      0 --:--:-- --:--:-- --:--:--  5928
  % Total    % Received % Xferd  Average Speed   Tim